# <font color=red>LangChain:  Memory</font>
- https://docs.langchain.com/docs

## What Does LangChain Provide?
+ Models
  + embedding
  + LLM (e.g. OpenAI)
+ Prompts
  + prompt templates
  + few-shot
  + example-selectors
  + output parsers
+ Chains (a multi-step workflow composed of <em>links</em>)</br>
  + Links (one of: prompt, model, another chain)
+ Vector Database Access
  + Document Loaders
  + Text Splitting 
<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
+ Memories (to facilitate chatbots or other 'iterative' sorts of apps)
</font></span>
+ Agents (loop over Thought, Act, Observe)
  + Tools
    + math
    + web search
    + custom (user-defined)

<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
## Memories
</font></span>
There are several types of memory available with LangChain.</br>
We will examine a simple version in a simple application, and then look at a couple of</br>
examples that use memory in concert with chains and/or agents.

First, let's do it <em>withOUT</em> memory.  We will use a plain LLMChain.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model_name="gpt-4", temperature=0, max_tokens=64)

template = """
You are a nice chatbot having a conversation with a human.
New human question: {question}
Response:
"""

prompt = PromptTemplate.from_template(template)

conversation = LLMChain(llm=llm, prompt=prompt, verbose=False)

response = conversation({"question": "hi, my name is Gizmo"})
print(response["text"])

response = conversation({"question": "what is the name of the first USA president?"})
print(response["text"])

## NOTE: we expect that the llm will NOT remember us
response = conversation({"question": "do you recall my name?"})
print(response["text"])

Next, let's do it <em>with</em> memory.  We will continue to use a plain LLMChain.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(model_name="gpt-4", temperature=0, max_tokens=64)

# NOTE: "chat_history" in the prompt template will be set by retrieving values from memory
template = """
You are a nice chatbot having a conversation with a human.
Previous conversation:
{chat_history}
New human question: {question}
Response:
"""

prompt = PromptTemplate.from_template(template)

# this is where we tell the memory that it maintains the value of chat_history
memory = ConversationBufferMemory(memory_key="chat_history")
# and we now apply the memory in the chain
conversation = LLMChain(llm=llm, prompt=prompt, verbose=False, memory=memory)

# we just pass in question - chat_history gets populated by memory
response = conversation({"question": "hi, my name is Gizmo"})
print(response["text"])

response = conversation({"question": "what was the name of the first USA president?"})
print(response["text"])

# the llm should remember us now
response = conversation({"question": "do you recall my name?"})
print(response["text"])

Now, we will get a little fancier, using a built-in ConversationChain.</br>
We will switch to a <font color=green>ConversationSummaryMemory</font> form of memory.</br>
This <em>summary</em> form of memory just maintains a summary of the content, to conserve memory.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationSummaryMemory

llm = ChatOpenAI(model_name="gpt-4", temperature=0, max_tokens=256)

summary_memory = ConversationSummaryMemory(llm=llm)

conversation = ConversationChain(llm=llm, verbose=False, memory=summary_memory)

answer = conversation.predict(input="Hi there! I am Gizmo.")
print(answer)

answer = conversation.predict(input="What is the name of the first USA president?")
print(answer)

answer = conversation.predict(input="Do you remember my name?")
print(answer)

print("----")
print(conversation.memory.buffer)